In [2]:
#匯入函式資料庫
#!pip install twstock
#!pip install yfinance
import twstock as t 
import pandas as p 
from bs4 import BeautifulSoup 
import requests 
import time as ti
import os as o


print("系統初始設定進行中...(0%)")

def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容
def post_data(message, token):#Line notify傳訊息
    try:
        url = "https://notify-api.line.me/api/notify"#api網址
        headers = {
            'Authorization': f'Bearer {token}'
        }
        payload = {
            'message': message
        }
        response = requests.request(
            "POST",
            url,
            headers=headers,
            data=payload
        )
        if response.status_code == 200:
            print(f"Success -> {response.text}")
    except Exception as _:
        print(_)#狀態

o.system( 'cls' )
print("系統取得上市股清單中...(10%)")
#####取得上市股#####
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)

o.system( 'cls' )
print("系統取得上櫃股清單中...(25%)")
#####取得上櫃股#####
data2 = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4', encoding='big5hkscs', header=0)[0]
data2 = p.DataFrame(data2["有價證券代號及名稱"])
data2 = data2["有價證券代號及名稱"].astype(str)

o.system( 'cls' )
print("系統取得可當沖股清單中...(30%)")
#####取得可當沖股#####
TW50 = [1101, 1216, 1301, 1303, 1326, 1590, 2002, 2207, 2227, 2303, 2308, 2317, 2327, 2330, 2352, 2357, 2379, 2382, 2395, 2408, 2409, 2412, 2454, 2603, 2609, 2610, 2615, 2801, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2891, 2892, 2912, 3008, 3034, 3037, 3045, 3711, 4904, 5871, 5876, 5880, 6415, 6505, 8046, 8454, 9910]
RICH50 = [1565, 1785, 3081, 3105, 3152, 3211, 3218, 3227, 3228, 3260, 3264, 3293, 3324, 3374, 3529, 3552, 3680, 3707, 4105, 4123, 4128, 4736, 4966, 5274, 5289, 5347, 5351, 5371, 5425, 5483, 6104, 6121, 6138, 6147, 6182, 6274, 6411, 6488, 6510, 6547, 6548, 6561, 6732, 6741, 8069, 8086, 8255, 8299, 8358, 8436]
MID100A = [1102,1210,1227,1229,1402,1434,1440,1451,1476,1477,1504,1560,1605,1717,1718,1722,1802,1907,2006,2014,2023,2027,2049,2105,2201,2301,2312,2313,2324,2337,2344,2347,2353,2354,2356,2360,2371,2376,2377,2385,2441,2449,2458,2474,2478,2492,2498,2542,2606,2618,2633,2637,2809,2812,2834,2845,2888,2889,2890,2915,3023,3035,3036,]
MID100B = [3044,3051,3189,3231,3406,3443,3481,3532,3533,3653,3661,3702,3714,4137,4919,4938,4958,4961,5269,5522,6005,6116,6176,6213,6239,6271,6409,6531,6592,6669,6719,6770,6781,8464,9904,9914,9917,9921,9941,9945]
Ulti_list = RICH50  + MID100A + MID100B + TW50 

o.system( 'cls' )
print("系統創建空串列中...(40%)")
#####創建空串列#####
stocklist=[]#篩選股票清單
stock28list = []#符合條件清單
amu28list = []#符合精選條件清單
ulti28list = []#符合當沖精選條件清單

o.system( 'cls' )
print("系統代數預設中...(50%)")
#####代數預設指定#####
d1s = 1 #上市股開始index碼
d1e = 968 #上市股結束index碼
d2s = 7408 #上櫃股開始index碼
d2e = 8205 #上櫃股結束index碼

o.system( 'cls' )
print("系統開始設定index值...(60%)")
fast_start = int(input("冷啟動請輸入0|快啟動請輸入1:"+"\n上市股始/終:"+str(d1s)+"/"+str(d1e)+"\n上櫃股始/終:"+str(d2s)+"/"+str(d2e)+"\n"))

o.system( 'cls' )
print("系統開始設定index值...(65%)")
#####檢查上市股代數變動#####
check_data_index = int(input("請問上市股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d1s)+"/"+str(d1e)))

o.system( 'cls' )
print("系統開始設定index值...(67%)")
if check_data_index == 0 and fast_start == 0:
  o.system( 'cls' )
elif check_data_index == 1 and fast_start == 0:
  d1s = int(input("請輸入上市股起始index:"))
  o.system( 'cls' )
  print("系統開始設定index值...(72%)")
  d1e = int(input("請輸入上市股終止index:"))
  o.system( 'cls' )
  print("系統開始設定index值...(76%)")
  
o.system( 'cls' )
print("系統開始設定index值...(77%)")
#####檢查上櫃股代數變動#####
check_data2_index = int(input("請問上櫃股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d2s)+"/"+str(d2e)))

o.system( 'cls' )
print("系統開始設定index值...(80%)")
if check_data2_index == 0 and fast_start == 0:
  o.system( 'cls' )
elif check_data2_index == 1 and fast_start == 0:
  d2s = int(input("請輸入上櫃股起始index:"))
  o.system( 'cls' )
  print("系統開始設定index值...(85%)")
  d2e = int(input("請輸入上櫃股終止index:"))
  o.system( 'cls' )
  print("系統開始設定index值...(89%)")

o.system( 'cls' )
print("系統製作上市股篩選股票清單...(90%)")
#####匯入上市股至篩選股票清單#####
for x in data.iloc[d1s:d1e]:#個別抓資料
  try:
    stockid = x
    stocklist.append(stockid)#匯入清單
  except:
    continue
o.system( 'cls' )
print("系統製作上櫃股篩選股票清單...(95%)")
#####匯入上櫃股至篩選股票清單#####
for z in data2.iloc[d2s:d2e]:#個別抓資料
  try:
    stockid = z
    stocklist.append(stockid)#匯入清單
  except:
    continue

o.system( 'cls' )
print("系統設定完成...(100%)\n開始篩選股票\n===============================")
print("實時符合當沖精選條件的股票:")
#####篩選股票迴圈#####
for x in stocklist:#個別抓資料
    try:
        ti.sleep(15)#待機15秒
        stk = x[:4]#取資料前4碼
        #####查詢最新股價#####
        price_url = 'https://tw.stock.yahoo.com/quote/' + stk #查詢股價的網址
        systime = ti.localtime()#取得查價時間
        span = web(price_url).find_all("span")#抓span資料
        for p in span:
            if p.text == '成交':#抓成交sapn
                price = float(span[span.index(p) + 1].string) #股價sapn
                lastprice = price#給值
            else:
                continue
        #########################
        #####取得個股各項資料#####
        stock = t.Stock(stk) #送出查詢至twse並取得回傳資料
        date = stock.date #從回傳資料取日期並給值
        price = stock.price#從回傳資料取價格並給值
        #########################
        #####計算28MA#####
        average = stock.moving_average(price, 28)#取價並依序計算並給值
        last28price = average[3]#從值取第4位價格
        time = ti.strftime("%Y/%m/%d", systime)#重新編時間格式
        #########################
        #####查詢成交量#####
        amount = stock.capacity#從回傳資料取成交量並給值
        lastamu = amount[30]#從值取第31位成交量
        recentamu = amount[26:]#從值取後5日成交量
        maxamu = max(recentamu)#從值取後5日最大成交量
        #########################
        #####是否站上28均線#####
        if lastprice >= last28price:#是，站上28均線
            stock28list.append(x)#匯入資料至符合條件清單
            #####比較成交量#####
            if lastamu >= maxamu:#是，成交量大
                amu28list.append(x)#匯入資料至符合精選條件清單
                stk = int(stk)#轉換stk為整數
                #####是否為可當沖個股#####
                if stk in Ulti_list:#是
                    stk = str(stk)#轉換stk為文字
                    print(time)#輸出取價時間
                    print('股票代碼',stk)#輸出股票代碼
                    ulti28list.append(x)#匯入資料至符合當沖精選條件清單
                elif stk not in Ulti_list:
                    continue
                #########################
            #########################
            '''
            #####Line Notify通知#####
            if __name__ == "__main__":
                token = "bgTlyk4WuerqsMIDvCCQSaAJDXFaVRaiVQEZFIZN8ou" # Token
                message = {"\n📊股票名稱\n【"+ str(x) +"】\n💵最新價格\n【"+ str(lastprice) +" 元】\n"+ str(amust) +"\n🕰取價時間\n【"+ str(time)+"】"}     # 要發送的訊息
                post_data(message, token)
            #########################
            '''
        #########################
    except:
        continue
#########################
print(stock28list)#輸出符合條件清單
print("----------------------")
print(amu28list)#輸出符合精選條件清單
print("----------------------")
print(ulti28list)#輸出符合當沖精選條件清單
print("----------------------")
systime2 = ti.localtime()#取得結束時間
time2 = ti.strftime("%Y/%m/%d, %H:%M:%S", systime2)#重新編時間格式
'''#####Line Notify通知#####
if __name__ == "__main__":
    token = "bgTlyk4WuerqsMIDvCCQSaAJDXFaVRaiVQEZFIZN8ou" # Token
    message = {"\n⚙系統一已完成股票篩選\n🕰系統時間\n【"+ str(time2)+"】\n符合條件之股票清單\n"+str(stock28list)}     # 要發送的訊息
    post_data(message, token)
#########################'''
#####Line Notify通知#####
if __name__ == "__main__":
    token = "bgTlyk4WuerqsMIDvCCQSaAJDXFaVRaiVQEZFIZN8ou" # Token
    message = {"\n⚙系統已完成股票篩選\n🕰系統時間\n【"+ str(time2)+"】\n符合當沖精選條件之股票清單\n"+str(ulti28list)}     # 要發送的訊息
    post_data(message, token)
#########################

系統初始設定進行中...(0%)
系統取得上市股清單中...(10%)
系統取得上櫃股清單中...(25%)
系統取得可當沖股清單中...(30%)
系統創建空串列中...(40%)
系統代數預設中...(50%)
系統開始設定index值...(60%)
系統開始設定index值...(65%)
系統開始設定index值...(67%)
系統開始設定index值...(77%)
系統開始設定index值...(80%)
系統開始設定index值...(85%)
系統開始設定index值...(89%)
系統製作上市股篩選股票清單...(90%)
系統製作上櫃股篩選股票清單...(95%)
系統設定完成...(100%)
開始篩選股票
實時符合當沖精選條件的股票:
2022/06/17
股票代碼 1301
2022/06/17
股票代碼 1440
2022/06/17
股票代碼 1590
2022/06/17
股票代碼 2006
2022/06/17
股票代碼 2049
2022/06/17
股票代碼 2352
2022/06/17
股票代碼 2382
2022/06/17
股票代碼 2633
2022/06/17
股票代碼 2812
2022/06/17
股票代碼 2889
2022/06/17
股票代碼 3036
2022/06/17
股票代碼 3044
2022/06/17
股票代碼 3231
2022/06/17
股票代碼 3702
2022/06/17
股票代碼 4904
2022/06/17
股票代碼 4938
2022/06/17
股票代碼 6176
2022/06/18
股票代碼 6505
2022/06/18
股票代碼 9904
2022/06/18
股票代碼 9910
2022/06/18
股票代碼 4105
2022/06/18
股票代碼 8069
['1104\u3000環泥', '1203\u3000味王', '1218\u3000泰山', '1225\u3000福懋油', '1231\u3000聯華食', '1232\u3000大統益', '1234\u3000黑松', '1235\u3000興泰', '1236\u3000宏亞', '1256\u3000鮮活果汁-KY', '1301\u3000台塑', '1304\u3000台聚', 